In [21]:
import pandas as pd
import numpy as np
import json
from itertools import chain

In [22]:
provider = "cloudflare"
target = "single"
h2_open = open(f"../log-{target}-{provider}" + "/h2-performances.json", "r")
h3_open = open(f"../log-{target}-{provider}" + "/h3-performances.json", "r")

h2_load = json.load(h2_open)
h3_load = json.load(h3_open)

h2_open.close()
h3_open.close()

h2_entrieses = [
    [
        {"number": entries["number"], "start": entries["start"], **entry}
        for entry in entries["entries"]
    ]
    for entries in h2_load
]
h3_entrieses = [
    [
        {"number": entries["number"], "start": entries["start"], **entry}
        for entry in entries["entries"]
    ]
    for entries in h3_load
]


h2_entries = pd.DataFrame(sum(h2_entrieses, []))
h3_entries = pd.DataFrame(sum(h3_entrieses, []))

h2_entries = h2_entries[-h2_entries['name'].str.contains('cdn-cgi/scripts')]
h3_entries = h3_entries[-h3_entries['name'].str.contains('cdn-cgi/scripts')]

h2_grouby_number = h2_entries.select_dtypes([float, int]).groupby("number")
h2_grouby_number_count = h2_grouby_number["start"].count()
h2_grouby_number_mode = h2_grouby_number_count.mode()[0]
h2_grouby_number_suc = h2_grouby_number_count == h2_grouby_number_mode
h2_grouby_number_min = h2_grouby_number["connectStart"].min()[h2_grouby_number_suc]
h2_grouby_number_max = h2_grouby_number["responseEnd"].max()[h2_grouby_number_suc]

h2_grouby_number_duration = h2_grouby_number_max - h2_grouby_number_min

h3_grouby_number = h3_entries.select_dtypes([float, int]).groupby("number")
h3_grouby_number_count = h3_grouby_number["start"].count()
h3_grouby_number_mode = h3_grouby_number_count.mode()[0]
h3_grouby_number_suc = h3_grouby_number_count == h3_grouby_number_mode
h3_grouby_number_min = h3_grouby_number["connectStart"].min()[h3_grouby_number_suc]
h3_grouby_number_max = h3_grouby_number["responseEnd"].max()[h3_grouby_number_suc]

h3_grouby_number_duration = h3_grouby_number_max - h3_grouby_number_min

h3_grouby_number_duration.median() / h2_grouby_number_duration.median()

0.9987647785424387

In [23]:
h2_entries.head()

,number,start,name,entryType,startTime,duration,initiatorType,nextHopProtocol,renderBlockingStatus,workerStart,...,unloadEventEnd,domInteractive,domContentLoadedEventStart,domContentLoadedEventEnd,domComplete,loadEventStart,loadEventEnd,type,redirectCount,activationStart
0,0,1.675529e+12,https://site.totto.page/single.html,navigation,0.0,1513.0,navigation,h2,blocking,0,...,0.0,213.7,247.2,249.5,1508.5,1508.5,1513.0,navigate,0.0,0.0
1,0,1.675529e+12,https://site.totto.page/css/4k.css,resource,167.2,81.3,link,h2,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1.675529e+12,https://site.totto.page/css/8k.css,resource,167.7,89.1,link,h2,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1.675529e+12,https://site.totto.page/css/12k.css,resource,168.1,264.6,link,h2,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1.675529e+12,https://site.totto.page/css/16k.css,resource,168.3,415.3,link,h2,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
h3_entries.head()

,number,start,name,entryType,startTime,duration,initiatorType,nextHopProtocol,renderBlockingStatus,workerStart,...,unloadEventEnd,domInteractive,domContentLoadedEventStart,domContentLoadedEventEnd,domComplete,loadEventStart,loadEventEnd,type,redirectCount,activationStart
0,0,1.675532e+12,https://site.totto.page/single.html,navigation,0.0,1484.2,navigation,h3,blocking,0,...,0.0,187.3,471.6,473.1,1482.8,1482.8,1484.2,navigate,0.0,0.0
1,0,1.675532e+12,https://site.totto.page/css/4k.css,resource,149.4,35.2,link,h3,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1.675532e+12,https://site.totto.page/css/8k.css,resource,149.7,39.2,link,h3,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1.675532e+12,https://site.totto.page/css/12k.css,resource,149.9,101.2,link,h3,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1.675532e+12,https://site.totto.page/css/16k.css,resource,150.2,125.9,link,h3,non-blocking,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
h2_entries["connect_duration"] = h2_entries["connectEnd"] - h2_entries["connectStart"]
h2_entries["response_duration"] = (
    h2_entries["responseEnd"] - h2_entries["requestStart"]
)
h2_entries[["name", "connect_duration", "response_duration"]]


,name,connect_duration,response_duration
0,https://site.totto.page/single.html,60.4,66.4
1,https://site.totto.page/css/4k.css,0.0,79.3
2,https://site.totto.page/css/8k.css,0.0,86.3
3,https://site.totto.page/css/12k.css,0.0,260.7
4,https://site.totto.page/css/16k.css,0.0,411.4
...,...,...,...
51995,https://site.totto.page/js/10k.js,0.0,398.5
51996,https://site.totto.page/js/8k.js,0.0,354.2
51997,https://site.totto.page/js/6k.js,0.0,309.2
51998,https://site.totto.page/js/4k.js,0.0,261.8


In [26]:
h3_entries["connect_duration"] = h3_entries["connectEnd"] - h3_entries["connectStart"]
h3_entries["response_duration"] = (
    h3_entries["responseEnd"] - h3_entries["requestStart"]
)
h3_entries[["name", "connect_duration", "response_duration"]]

,name,connect_duration,response_duration
0,https://site.totto.page/single.html,39.6,65.2
1,https://site.totto.page/css/4k.css,0.0,34.2
2,https://site.totto.page/css/8k.css,0.0,37.9
3,https://site.totto.page/css/12k.css,0.0,99.0
4,https://site.totto.page/css/16k.css,0.0,122.7
...,...,...,...
51995,https://site.totto.page/js/10k.js,0.0,299.6
51996,https://site.totto.page/js/8k.js,0.0,256.7
51997,https://site.totto.page/js/6k.js,0.0,210.7
51998,https://site.totto.page/js/4k.js,0.0,172.5


In [27]:
h2_grouby_name = h2_entries[["name", "connect_duration", "response_duration"]].groupby("name")
h2_grouby_name_mean = h2_grouby_name.mean()
h2_grouby_name_mean_sum = h2_grouby_name_mean.sum()
h3_grouby_name = h3_entries[["name", "connect_duration", "response_duration"]].groupby("name")
h3_grouby_name_mean = h3_grouby_name.mean()
h3_grouby_name_mean_sum = h3_grouby_name_mean.sum()

In [28]:
def connection_duration(provider: str, target: str):
    h2_open = open(f"../log-{target}-{provider}" + "/h2-performances.json", "r")
    h3_open = open(f"../log-{target}-{provider}" + "/h3-performances.json", "r")

    h2_load = json.load(h2_open)
    h3_load = json.load(h3_open)

    h2_open.close()
    h3_open.close()

    h2_entrieses = [
        [
            {"number": entries["number"], "start": entries["start"], **entry}
            for entry in entries["entries"]
        ]
        for entries in h2_load
    ]
    h3_entrieses = [
        [
            {"number": entries["number"], "start": entries["start"], **entry}
            for entry in entries["entries"]
        ]
        for entries in h3_load
    ]
    h2_entries = pd.DataFrame(sum(h2_entrieses, []))
    h3_entries = pd.DataFrame(sum(h3_entrieses, []))

    h2_entries = h2_entries[-h2_entries["name"].str.contains("cdn-cgi/scripts")]
    h3_entries = h3_entries[-h3_entries["name"].str.contains("cdn-cgi/scripts")]

    url = h2_entries[0:1]["name"][0]
    h2_filtered = h2_entries[h2_entries["name"] == url].copy().reset_index(drop=True)
    h3_filtered = h3_entries[h3_entries["name"] == url].copy().reset_index(drop=True)

    h2_connect_duration = h2_filtered["connectEnd"] - h2_filtered["connectStart"]
    h3_connect_duration = h3_filtered["connectEnd"] - h3_filtered["connectStart"]

    h2_lower_limit = h2_connect_duration.mean() - h2_connect_duration.std() * 3
    h2_upper_limit = h2_connect_duration.mean() + h2_connect_duration.std() * 3
    h3_lower_limit = h3_connect_duration.mean() - h3_connect_duration.std() * 3
    h3_upper_limit = h3_connect_duration.mean() + h3_connect_duration.std() * 3

    h2_connect_duration = (
        h2_connect_duration[
            (h2_lower_limit < h2_connect_duration)
            & (h2_connect_duration < h2_upper_limit)
        ]
        .copy()
        .reset_index(drop=True)
    )
    h3_connect_duration = (
        h3_connect_duration[
            (h3_lower_limit < h3_connect_duration)
            & (h3_connect_duration < h3_upper_limit)
        ]
        .copy()
        .reset_index(drop=True)
    )

    h2_response_duration = h2_filtered["responseEnd"] - h2_filtered["requestStart"]
    h3_response_duration = h3_filtered["responseEnd"] - h3_filtered["requestStart"]

    h2_lower_limit = h2_response_duration.mean() - h2_response_duration.std() * 3
    h2_upper_limit = h2_response_duration.mean() + h2_response_duration.std() * 3
    h3_lower_limit = h3_response_duration.mean() - h3_response_duration.std() * 3
    h3_upper_limit = h3_response_duration.mean() + h3_response_duration.std() * 3

    h2_response_duration = (
        h2_response_duration[
            (h2_lower_limit < h2_response_duration)
            & (h2_response_duration < h2_upper_limit)
        ]
        .copy()
        .reset_index(drop=True)
    )
    h3_response_duration = (
        h3_response_duration[
            (h3_lower_limit < h3_response_duration)
            & (h3_response_duration < h3_upper_limit)
        ]
        .copy()
        .reset_index(drop=True)
    )

    result = pd.DataFrame()
    result["h2_connect"] = h2_connect_duration
    result["h3_connect"] = h3_connect_duration
    result["h2_response"] = h2_response_duration
    result["h3_response"] = h3_response_duration
    result["connect_duration"] = h3_connect_duration / h2_connect_duration
    result["response_duration"] = h3_response_duration / h2_response_duration
    result = result.dropna()

    return result, url


In [30]:
for provider, target in [("client1", "simple"), ("client1", "single")]:
    tmp, name = connection_duration(provider=provider, target=target)
    print(
        name,
        tmp["h2_connect"].mean()
        / (tmp["h2_connect"].mean() + tmp["h2_response"].mean()),
        tmp["h2_connect"].mean(),
        tmp["h2_response"].mean(),
        tmp["h3_connect"].mean()
        / (tmp["h3_connect"].mean() + tmp["h3_response"].mean()),
        tmp["h3_connect"].mean(),
        tmp["h3_response"].mean(),
    )


https://quic.totto.page/simple.html 0.3821222412560843 14.669021190145061 23.71927346113231 0.40356243889428584 16.08133198725347 23.767103935857833
https://quic.totto.page/single.html 0.37230949708431726 14.791286726853043 24.937183383871115 0.3918233326378069 16.080040527000012 24.958966565168375
